# Final Assignment CSE 368
prwilkin, adong4, duncenzh

**Citations:**
---
Environment 1: 
> Dataset is curated/modified; US Department of Transportation Federal Highway Administration Route Log and Finder List “Table 4: Major Cities Served by Interstate Routes (2010 Census Populations greater than 50,000)”,

> NetworkX library; Aric A. Hagberg, Daniel A. Schult and Pieter J. Swart, “Exploring network structure, dynamics, and function using NetworkX”, in Proceedings of the 7th Python in Science Conference (SciPy2008), Gäel Varoquaux, Travis Vaught, and Jarrod Millman (Eds), (Pasadena, CA USA), pp. 11–15, Aug 2008,

> Pandas library; McKinney, W., & others. (2010). Data structures for statistical computing in python. In Proceedings of the 9th Python in Science Conference (Vol. 445, pp. 51–56),

Environment 2:
> Numpy library; Harris, C.R., Millman, K.J., van der Walt, S.J. et al. Array programming with NumPy. Nature 585, 357–362 (2020). DOI: 10.1038/s41586-020-2649-2,

> Classes State, Node, & Problem modified; fall23 cse368 assignment 1 base code provided by Professor Alina Vereshchaka and TA’s

# Environment 1

In [428]:
import pandas as pd
import networkx as nx


def generate(datafile, graph):
    for index, row in datafile.iterrows():  
        # Node will be named after 'city, state'
        node_name = f"{row['city']}, {row['state']}"
        # Parse lists
        connected_cities = row['connected cities'].split(',')
        connected_states = row['connected cities states'].split(',')
        distances = [int(dist) for dist in row['distance to city'].split(',')]

        # Add the current city if not already in graph or update with lat, long, & interstates
        graph.add_node(node_name, lat=row['lat'], long=row['long'], interstates=row['interstates'])

        # Add edges
        for i, (connected_city, connected_state) in enumerate(zip(connected_cities, connected_states)):
            connected_city_state = f"{connected_city.strip()}, {connected_state.strip()}"
            if connected_city_state not in graph:
                graph.add_node(connected_city_state)
            # Distance is cost
            graph.add_edge(node_name, connected_city_state, distance=distances[i])


# visual code goes here

In [429]:
from queue import PriorityQueue

# Load dataset
file = './data.xlsx'
data = pd.read_excel(file)

# Initialize graph
G = nx.Graph()

def bfs(start_city, end_city):
    counter = 0
    queue = []
    visited = []
    route = []
    previous_cities = []

    queue.append(start_city)

    while queue:
        current_city = queue.pop(0)
        visited.append(current_city)
        counter += 1

        if current_city == end_city:
            route.append(end_city)
            # route creating
            city = end_city
            for i in range(counter):  # amount of times loopings needed to be done (not too sure if needed)
                for cities in previous_cities:  # loops through list of city to check
                    if city == cities[0]:  # matches the city to find the previous city
                        route.append(cities[1])  # adds previous city to the route
                        city = cities[1]  # updates the city to backtrack
            # route.reverse()
            # print(route)
            # print(counter)
            return route

        for connected_cities in G.edges(current_city):
            if connected_cities[1] not in visited:
                queue.append(connected_cities[1])
                previous_cities.append([connected_cities[1], current_city])  # to help backtrack later on
                

def A_star(start_city, end_city):
    priority_queue = PriorityQueue()
    visited = set()
    route = []
    previous_cities = []
    counter = 0
    start_node_tuple = (0, start_city)

    list_of_connected_cities_and_distance = []
    for edge in G.edges.data():
        list_of_connected_cities_and_distance.append(list([edge[0], edge[1], int(edge[2]['distance'])]))

    priority_queue.put(start_node_tuple)

    while priority_queue:
        counter += 1
        tup = priority_queue.get()
        distance, parent_city = tup[0], tup[1]
        visited.add(parent_city)
        if parent_city == end_city:
            route.append(end_city)

            # route creating
            city = end_city
            for i in range(counter):  # amount of times loopings needed to be done (not too sure if needed)
                for the_cities in previous_cities:  # loops through list of city to check
                    if city == the_cities[1]:  # matches the city to find the previous city
                        route.append(the_cities[0])  # adds previous city to the route
                        city = the_cities[0]  # updates the city to backtrack
            # route.reverse()
            # print(route)
            # print(distance)
            # print(counter)
            return route, distance, counter

        cities = []
        for connected_cities in G.edges(parent_city):
            for find_distance in list_of_connected_cities_and_distance:
                if connected_cities[0] in find_distance and connected_cities[1] in find_distance:
                    distance_of_connected_cities = find_distance[2]
            if connected_cities[1] not in visited:
                tuple_placeholder = (distance_of_connected_cities + distance, connected_cities[1])
                cities.append(tuple_placeholder)
                child_tup = tuple_placeholder
                priority_queue.put(child_tup)
                newcity, newdistance = child_tup[1], child_tup[0]
                previous_cities.append((parent_city, newcity))


In [430]:
from env2 import State, Node, Problem, childNode
import time

def dijkstra(problem):
    starttime = time.time()
    counter = 0
    queue = []
    visited = []
    dist = {}
    start = Node(None, None, 0, problem.initialState, 0)
    dist[problem.initialState.toTuple()] = [0,start]
    queue.append(start)
    while queue:
        counter +=1
        parent = queue.pop(0)
        depth = parent.depth
        if problem.goalTest(parent.state):
            print(str(counter) + '\n')
            print(parent.state)
            """dpath = []
            dnode = parent
            while dnode.parent != None:
                dpath.append(dnode.action)
                found = False
                for action in problem.applicable(dnode.state):
                    print('yes')
                    child = childNode(parent, action, problem, depth)
                    if child.state.toTuple() in dist:
                        print('yes2')
                        if dist[child.state.toTuple()][0] == dist[dnode.state.toTuple()][0] - 1:
                            dnode = dist[child.state.toTuple()][1]
                            found = True
                            print('yes3')
                            break
                if not found:
                    break
            dpath.reverse()
            print('test: ',dpath) """
            path = []
            node = parent
            while node.parent != None:
                path.append(node.action)
                node = node.parent
            path.reverse()
            print(path)
            endtime = time.time() - starttime
            return counter, parent.state, path, endtime
        visited.append(parent.state)
        for action in problem.applicable(parent.state):
            child = childNode(parent, action, problem, depth)
            if child.state not in visited and child not in queue:
                queue.append(child)
                if child.state.toTuple() in dist:
                    if dist[child.state.toTuple()][0] > dist[parent.state.toTuple()][0] + child.cost:
                        dist[child.state.toTuple()] = [dist[parent.state.toTuple()][0] + child.cost, child]
                else:
                    dist[child.state.toTuple()] = [dist[parent.state.toTuple()][0] + child.cost, child]
        print(str(counter) + "\n", parent.state)


test = State()
problem = Problem()
problem.initialState = State(test)
dijkstra(problem)



1
 [2 1 1 1 1 1 1 1 1 1]
[1 0 1 1 1 1 1 1 0 0]
[1 1 0 1 0 1 0 1 1 1]
[1 1 0 1 1 1 1 1 0 1]
[1 1 1 1 0 1 0 1 0 3]
[0 0 0 1 1 1 1 1 1 1]
[1 1 1 1 1 0 1 1 0 1]
[0 0 0 1 1 0 1 1 1 1]
[1 1 0 1 0 0 0 1 0 0]
[1 1 1 1 1 1 1 1 1 1]

2
 [1 1 1 1 1 1 1 1 1 1]
[2 0 1 1 1 1 1 1 0 0]
[1 1 0 1 0 1 0 1 1 1]
[1 1 0 1 1 1 1 1 0 1]
[1 1 1 1 0 1 0 1 0 3]
[0 0 0 1 1 1 1 1 1 1]
[1 1 1 1 1 0 1 1 0 1]
[0 0 0 1 1 0 1 1 1 1]
[1 1 0 1 0 0 0 1 0 0]
[1 1 1 1 1 1 1 1 1 1]

3
 [1 2 1 1 1 1 1 1 1 1]
[1 0 1 1 1 1 1 1 0 0]
[1 1 0 1 0 1 0 1 1 1]
[1 1 0 1 1 1 1 1 0 1]
[1 1 1 1 0 1 0 1 0 3]
[0 0 0 1 1 1 1 1 1 1]
[1 1 1 1 1 0 1 1 0 1]
[0 0 0 1 1 0 1 1 1 1]
[1 1 0 1 0 0 0 1 0 0]
[1 1 1 1 1 1 1 1 1 1]

4
 [1 1 1 1 1 1 1 1 1 1]
[1 0 1 1 1 1 1 1 0 0]
[2 1 0 1 0 1 0 1 1 1]
[1 1 0 1 1 1 1 1 0 1]
[1 1 1 1 0 1 0 1 0 3]
[0 0 0 1 1 1 1 1 1 1]
[1 1 1 1 1 0 1 1 0 1]
[0 0 0 1 1 0 1 1 1 1]
[1 1 0 1 0 0 0 1 0 0]
[1 1 1 1 1 1 1 1 1 1]

5
 [1 1 2 1 1 1 1 1 1 1]
[1 0 1 1 1 1 1 1 0 0]
[1 1 0 1 0 1 0 1 1 1]
[1 1 0 1 1 1 1 1 0 1]
[1 1 1 1 0 1 

(253,
 ['D', 'D', 'D', 'D', 'R', 'R', 'R', 'D', 'D', 'D', 'D', 'D', 'R'],
 0.31230902671813965)